# Usage Example of the HAL Project Group
## Prerequisite: 
- Importing required libraries, 
- Setting system path
- Importing HAL project

In [1]:
import os
import sys

In [ ]:
pip install plotly

In [2]:
sys.path.append(os.getcwd() + "/hal_pm")

In [4]:
from hal_pm import *
import datetime

## 1.) & 2.): Etablishing connection to Rest-API and requesting the data
The load_data function takes the coordinates (latitude and logitude, start and end point each) as well as a start date in `datetime` format and number of delta hours as input values. 
In first place, PM2.5 and PM10 `pandas` data frames are created seperetally. Meanwhile it creates the unique identifier column *measurement_id*, which is a combination of *sensor_id* and *time* columns in each data frame. By the unique identifier, PM2.5 and PM10 data frames are merged into a large one (since a single sensor always measures PM10 and PM2.5 values). After reordering the columns, the large data frame containing all *measurements*, *dates*, *coordinates* and *sensor_id*s is outputted to the user.

#### Place and time of measurements:
As a group, we decided to take a look at Stuttgart's fine particulate pollution in our example. This is reasoned on in the fact, that Stuttgart were famous for its ban on diesel-powered vehicles in an early stage compared to the rest of germany. Due to the fact, that one hour already contains 20134 observations, we couldn't take a longer time period into account (otherwise, loading data would take too much time). Therefore *start_date* is chosen without purpose as well. However, by only defining the datetime as *YYYY, MM, DD*, the measurements start on midnight 2018-06-01 and end on 1:00 am 2018-06-01.

In [5]:
# Load Data
start_date = datetime.datetime(2018, 6, 1)
pmdata = Load_Data.load_data(lat_start = 48.5, lat_end = 49, long_start = 9, long_end = 9.3, start_datetime = start_date, delta_hours = 1)

In [6]:
pmdata

,measurement_PM10,measurement_PM2.5,time,lat,lon,sensor_id,measurement_id
0,NaN,NaN,2018-06-01T00:00:01Z,48.798,9.070,11264,11264_2018-06-01T00:00:01Z
1,2.67,1.27,2018-06-01T00:00:01Z,48.949,9.098,122,122_2018-06-01T00:00:01Z
2,8.60,1.77,2018-06-01T00:00:01Z,48.765,9.147,4383,4383_2018-06-01T00:00:01Z
3,12.35,2.42,2018-06-01T00:00:01Z,48.779,9.034,181,181_2018-06-01T00:00:01Z
4,NaN,NaN,2018-06-01T00:00:01Z,48.779,9.034,182,182_2018-06-01T00:00:01Z
...,...,...,...,...,...,...,...
20129,3.90,1.33,2018-06-01T00:59:59Z,48.759,9.162,309,309_2018-06-01T00:59:59Z
20130,NaN,NaN,2018-06-01T00:59:59Z,48.536,9.274,202,202_2018-06-01T00:59:59Z
20131,NaN,NaN,2018-06-01T00:59:59Z,48.692,9.148,2493,2493_2018-06-01T00:59:59Z
20132,NaN,NaN,2018-06-01T00:59:59Z,48.799,9.224,5356,5356_2018-06-01T00:59:59Z


## 3.) Data Filtering
These functions roughly clean the data. That is, they eliminate observations with missing values as well as outlier observations following a user-specified option. As soon as either a PM10 or a PM2.5 measurement is missing or identified as an outlier, the function removes the entire observation. We chose this approach since we aim at calculating correlations between PM10 and PM2.5 measurements, thus we need valid data for both. Moreover, the amount of data is vast and assuming that outliers as well as missing values are random, we can remove the respective observations.

In [7]:
# Clear Data
pmdata = Filter_Data.remove_missing(pmdata)
pmdata = Filter_Data.remove_outliers(pmdata ,method = "Z-score")
print(pmdata)

9590 observations with missing values were removed from the data frame
56 outlier observations were deleted
       measurement_PM10  measurement_PM2.5                  time     lat  \
1                  2.67               1.27  2018-06-01T00:00:01Z  48.949   
2                  8.60               1.77  2018-06-01T00:00:01Z  48.765   
3                 12.35               2.42  2018-06-01T00:00:01Z  48.779   
5                  1.10               1.10  2018-06-01T00:00:01Z  48.791   
8                  5.12               2.78  2018-06-01T00:00:02Z  48.757   
...                 ...                ...                   ...     ...   
20119              5.10               1.15  2018-06-01T00:59:58Z  48.942   
20121              1.80               1.40  2018-06-01T00:59:58Z  48.870   
20124              3.23               1.80  2018-06-01T00:59:59Z  48.784   
20126             23.43              18.10  2018-06-01T00:59:59Z  48.516   
20129              3.90               1.33  2018-06-01T0

## 4.) Descriptive Statistics
These functions provide high-level descriptive statistics. `get_max` outputs a pandas dataframe containining the maximium oberservations for PM10 and Pm2.5 measurements. Moreover, the function provides an option to ouput a map with the location of the maxima. `plot_mean_pm` on the other hand outputs a plot with the mean PM10 and PM2.5 concentration over time in the study area.

In [ ]:
maxi, maxi_map = Descriptive_Stats.get_max(pmdata, get_map = True)


In [ ]:
maxi

In [ ]:
maxi_map

In [ ]:
Descriptive_Stats.plot_mean_pm(pmdata, time_interval = "5Min")

## 5.) Time histories of fine particulate pollution for the sensors with the highest and lowest mean pollution levels

The `plot_average_pol` function evaluates the *sensor_id*s with the lowest and highest average measurement per PM10/PM2.5 seperatelly. In a second steps, it takes the measurements of these sensors and plots them against a timeline. The plot is organized in subplots as following:
- **Rows**: PM10 or PM2.5 measurement
- **Columns**: sensors with average maximum (Red) or average minumum (Blue)

In case one is wondering why the average minimum time plot of PM10 measurment is a horizontal line: probably due to measurment errors, the sensor is measuring the same value all the time.

In [ ]:
# Plot average polution
Time_Plots.plot_average_pol(pmdata)

In [ ]:
min_PM10 = min(pmdata.groupby("sensor_id").mean()['measurement_PM10'])
min_PM10_id = pmdata.groupby("sensor_id").mean()[pmdata.groupby("sensor_id").mean()['measurement_PM10'] == min_PM10].index[0]
df_min_PM10 = pmdata.loc[pmdata['sensor_id'] == min_PM10_id]
print("Minimal Average PM10 Measurement:")
df_min_PM10

## 8.)  Mapping the data
The `map_data` function takes a `pandas` dataframe with PM10 andPM2.5 measurements as input an returns respective interactive maps which indicate the concentration of PM10 andPM2.5 in the study area. The time slider may be used to see concentration changes over time. Along the pandas dataframe, the user should provide a geojson file, which contains the geometries to which he or she would like to aggregate the data to. With the geojson file that we provide in this usage example, we can cover any coordinates in Germany. 

In [9]:
pm10_map = Map.map_data(df = pmdata, geo_boundaries = 'package/mapping_data/plz_ger.geojson',
                        measurement_type = "measurement_PM10", lat = 'lat', lon = 'lon', time_interval = "5Min")
pm25_map = Map.map_data(df = pmdata, geo_boundaries = 'package/mapping_data/plz_ger.geojson',
                        measurement_type = "measurement_PM2.5", lat = 'lat', lon = 'lon', time_interval = "5Min")

DriverError: package/mapping_data/plz_ger.geojson: No such file or directory

In [ ]:
pm10_map

In [ ]:
pm25_map